<a href="https://colab.research.google.com/github/nohjuhyeon/study_AIs/blob/main/docs/NLPs/08_NLP_classification_ynanewstitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 준비
- DDA, EDA 확인

In [1]:
import pandas as pd
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
topic = pd.read_csv("topic_dict.csv")

In [2]:
train[:2]

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4


In [3]:
test[:2]

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사


In [4]:
topic

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


### 데이터 전처리
- trainset, testset 병합
- tokenizing : 머신에 맞게 벡터화

In [5]:
raw_concat = pd.concat([train,test])
raw_concat.shape,train.shape,test.shape

((54785, 3), (45654, 3), (9131, 2))

In [6]:
raw_concat.columns

Index(['index', 'title', 'topic_idx'], dtype='object')

In [7]:
# 숫자 삭제
raw_concat['title_preprocess']= raw_concat['title'].str.replace('[0-9]','',regex=True)
# str.find() 하면 검색 가능

In [8]:
# 영문자 소문자로 통일
raw_concat['title_preprocess']=raw_concat['title_preprocess'].str.lower()

In [9]:
! pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 47.9 MB/s eta 0:00:00


In [10]:
from konlpy.tag import Okt
okt = Okt()

In [11]:
# # 조사, 어미 구두점 같은 것을 제거 함수 / 나중에 불용어 처리 함수도 작성해주는 것이 좋음
# def okt_clean(text):
#     clean_text = []
#     for word in okt.pos(text, stem=True):
#         if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
#           if word[0] in ['A-z'] or word[0] in ['ㄱ-힣']:
#               clean_text.append(word[0])

#     return " ".join(clean_text)

In [12]:
# 조사, 어미 구두점 같은 것을 제거 함수 / 나중에 불용어 처리 함수도 작성해주는 것이 좋음
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [13]:
raw_concat['title_okt_clean'] = raw_concat['title_preprocess'].apply(okt_clean)

In [14]:
raw_concat[:5]

,index,title,topic_idx,title_preprocess,title_okt_clean
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4.0,인천→핀란드 항공기 결항…휴가철 여행객 분통,인천 → 핀란드 항공기 결항 휴가 철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4.0,실리콘밸리 넘어서겠다…구글 조원 들여 美전역 거점화,실리콘밸리 넘어서다 구글 조원 들이다 美 전역 거점 화
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4.0,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4.0,nyt 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,nyt 클린턴 측근 韓 기업 특수 관계 조명 공과 사 맞다 물리다 종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4.0,시진핑 트럼프에 중미 무역협상 조속 타결 희망,시진핑 트럼프 중미 무역 협상 조속 타결 희망


In [15]:
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None)
# TfidVectorzier(tokenizer=None,ngram_range=(1,2))


In [16]:
tfidfVectorizer.fit(raw_concat['title_okt_clean'])

TfidfVectorizer()

In [17]:
# trainset, testset 분리
trainset = raw_concat[raw_concat['topic_idx'].notnull()] # train
testset = raw_concat[raw_concat['topic_idx'].isnull()] # test
trainset.shape, testset.shape

((45654, 5), (9131, 5))

## 모델

In [18]:
train_target  = trainset['topic_idx']
train_feature = tfidfVectorizer.transform(trainset['title_okt_clean']) # 머신 학습을 위한 선 작업

In [19]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier=RandomForestClassifier(n_jobs=-1)

In [20]:
randomForestClassifier.fit(train_feature,train_target)

RandomForestClassifier(n_jobs=-1)

## 평가

In [20]:
# randomForestClassifier.predict(train_feature[:1])
randomForestClassifier.predict(train_feature[120:125])

In [21]:
# train_target[:1]
train_target[120:125]

0    4.0
Name: topic_idx, dtype: float64

## 서비스

In [23]:
topic

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


In [35]:
# sentence = "첨단바이오 기술개발의 허브 ‘한국생명공학연구원’ [D:로그인]"
sentence = "서울 분양가 가장 높은 동네는…평당 4500만원 육박한 용산구"

In [36]:
# 형태소 분석(명사 기준)
sentence_nouns = okt_clean(sentence)
sentence_nouns

'서울 분양 가장 높다 동네 평당 4500만원 육박 용산구'

In [37]:
# Tokenizing
sentence_tokens = tfidfVectorizer.transform([sentence_nouns]) # 학습했을 때 형식 그대로 이용
sentence_tokens

<1x27288 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [38]:
sentence_tokens.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [39]:
import numpy as np
np.where(sentence_tokens.toarray()>0)

(array([0, 0, 0, 0, 0, 0, 0, 0]),
 array([ 1211,  5384,  6565, 11044, 12492, 17073, 17902, 24850]))

In [40]:
# 예측(predict)

In [41]:
randomForestClassifier.predict(sentence_tokens.toarray()) # 0 : IT 과학 / 1 : 경제

array([1.])

In [42]:
import pickle

In [43]:
with open('NLP_classification_ynanaewstitles.pkl','wb') as pickle_file:   # pickle에 저장
  pickle.dump(obj=randomForestClassifier,file=pickle_file)